# Métricas

El objetivo de este notebook es reunir las métricas que se obtienen a partir de las distintas técnicas de resuménes extractivos que se han 
desarrollado.

En esta primera parte, el objetivo ha sido realizar un resumen de los partidos, utilizando directamente los eventos. Por tanto, para evaluar
la bondad de estos métodos, se comparará directamente con la noticia asociada a cada partido, (la cual podemos considerar resumen del partido)
utilizando las siguientes métricas:

- ROUGE: mide el recall. Cuántas secuencias (unigramas, bigramas, lcs...) del resumen generado se encuentran en el resumen real, del total de las secuencias
del resumen real.
- BLEU: mide la precision. Cuántas secuencias del resumen generado se encuentran en el resumen real, del total de las secuencias del resumen generado.

Es importante considerar que no se está haciendo un resumen extractivo al uso (tradicionalmente se realizaría un resumen a partir de la noticia, 
y de ahí se extraerían las métricas). El estilo de escritura y los términos usados en eventos y noticias son bastante diferentes,
por lo que las métricas tradicionales obtenidas no serán demasiado altas. Nos interesa la comparación entre técnicas propuestas.